In [2]:
import tensorflow as tf
import os
from PIL import Image
import numpy as np

# Set your dataset path
dataset_path = 'archive/Lung Segmentation/masks'
processed_path = 'Mask-images'

# Function to process and save images
def preprocess_image(image_path, target_size=(224, 224), channels=3):
    try:
        img = tf.io.read_file(image_path)
        img = tf.io.decode_image(img, channels=channels, expand_animations=False)
        img = tf.image.resize(img, target_size)
        img = tf.cast(img, tf.float32) / 255.0  # Normalize to [0,1]
        
        # Remove any extra dimension if present
        img = tf.squeeze(img, axis=0) if img.shape.rank == 4 else img
        return img
    except tf.errors.InvalidArgumentError:
        print(f"Corrupted image detected and skipped: {image_path}")
        return None

# Process the dataset
def preprocess_dataset(dataset_path, processed_path):
    if not os.path.exists(processed_path):
        os.makedirs(processed_path)

    for subdir, dirs, files in os.walk(dataset_path):
        for file in files:
            file_path = os.path.join(subdir, file)
            image = preprocess_image(file_path)
            if image is not None:
                processed_file_path = os.path.join(processed_path, os.path.relpath(file_path, dataset_path))
                os.makedirs(os.path.dirname(processed_file_path), exist_ok=True)
                tf.keras.preprocessing.image.save_img(processed_file_path, image)

preprocess_dataset(dataset_path, processed_path)
